### کتابخانه های مورد نظر خود را اضافه می کنیم

In [1]:
import os
import nltk
from nltk.tag.stanford import StanfordNERTagger

import re
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

### ما مدل را از قبل بوسیله جاوا آماده کردیم و فقط در اینجا از آن استفاده می کنیم

In [2]:
os.environ['JAVAHOME'] = "C:/Program Files/Java/jre1.8.0_221/bin/java.exe"

jar = 'stanford-ner.jar'
model = 'trained_model.ser.gz'

ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

### فایل تست را میخوانیم و با استفاده از مدل ایجاد شده، لیبل های آنها را پیش بینی می کنیم

In [3]:
test_docs = [line.rstrip('\n') for line in open('Data/NERte.txt', encoding="utf8")]

true_labels = []
words = []
for i, doc in enumerate(test_docs):
    word = re.split(r'\t+', doc)
    if(word[0] != ""):
        words.append(word[0])
        true_labels.append(word[1])
        
        
predicts = ner_tagger.tag(words)
pred_labels = []
for element1, element2 in predicts:
    pred_labels.append(element2)

In [4]:
entity_counter = 0
all_entities_start_end_labels = []
for i, label in enumerate(true_labels[:]):
    if label != 'O':
        label_start_end = []
        label_start_end.append(true_labels[i])
        entity_counter += 1
        label_start_end.append(i)
        current_label = label
        j = i
        while current_label == label:
            j += 1
            current_label = true_labels[j]
        label_start_end.append(j)
        i = j
        
        all_entities_start_end_labels.append(label_start_end)
        
entity_counter_pred = 0
for i, label in enumerate(pred_labels[:]):
    if label != 'O':
        entity_counter_pred += 1
        current_label = label
        j = i
        while current_label == label:
            j += 1
            current_label = pred_labels[j]
        i = j

### دقت و ریکال را به صورت زیر بدست آوردیم

In [5]:
true_match_counter = 0
for i, element in enumerate(all_entities_start_end_labels):
    start = element[1]
    end = element[2]
    match = True
    for j in range(start, end):
        if element[0] != pred_labels[j]:
            match = False
    if match == True:
        true_match_counter += 1
        
print("Our Precision is: "+str(true_match_counter/entity_counter_pred))
print("Our Recall is: " + str(true_match_counter/entity_counter))

Our Precision is: 0.672972972972973
Our Recall is: 0.21899736147757257


### ماتریس سرگشتگی را به صورت زیر به دست آوردیم

In [6]:
all_true_labels = list(set(true_labels))

cm = confusion_matrix(true_labels, pred_labels, labels=all_true_labels)

#Normalization
np.set_printoptions(suppress=True)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

df = pd.DataFrame(cm, columns=all_true_labels, index=all_true_labels)
df

,other,person,O,movie,facility,geo-loc,tvshow,musicartist,sportsteam,company,product
other,0.122222,0.018519,0.848148,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.00000,0.000000
person,0.015038,0.421053,0.556391,0.000000,0.000000,0.007519,0.000000,0.000000,0.000000,0.00000,0.000000
O,0.001200,0.000991,0.997131,0.000104,0.000209,0.000261,0.000000,0.000000,0.000052,0.00000,0.000052
movie,0.000000,0.000000,0.842105,0.157895,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
facility,0.029126,0.019417,0.631068,0.000000,0.310680,0.009709,0.000000,0.000000,0.000000,0.00000,0.000000
geo-loc,0.000000,0.052288,0.627451,0.000000,0.065359,0.254902,0.000000,0.000000,0.000000,0.00000,0.000000
tvshow,0.000000,0.043478,0.869565,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.00000,0.000000
musicartist,0.000000,0.027397,0.917808,0.000000,0.000000,0.000000,0.000000,0.054795,0.000000,0.00000,0.000000
sportsteam,0.000000,0.000000,0.789474,0.000000,0.000000,0.052632,0.000000,0.000000,0.157895,0.00000,0.000000
company,0.000000,0.023256,0.720930,0.000000,0.034884,0.000000,0.000000,0.000000,0.000000,0.22093,0.000000


### همانطور که در زیر مشاهده می شود بیشترین خطا به دلیل استفاده از لیبل
### O
### به جای لیبل 
### musicartist, tvshow, other 
### می باشد

In [7]:
non_diagonal_values = []
non_diagonal_labels = []
for i in range(len(cm)):
    for j in range(len(cm)):
        if j == i:
            continue
        else:   
            non_diagonal_labels.append(tuple([str(all_true_labels[i])+","+str(all_true_labels[j]),cm[i][j]]))
            non_diagonal_values.append(cm[i][j])
            
sorted_list_of_non_diagonal = np.flipud(np.argsort(non_diagonal_values))

for i in range(len(sorted_list_of_non_diagonal[:10])):
    print(non_diagonal_labels[sorted_list_of_non_diagonal[i]])

('musicartist,O', 0.9178082191780822)
('tvshow,O', 0.8695652173913043)
('other,O', 0.8481481481481481)
('movie,O', 0.8421052631578947)
('product,O', 0.8301886792452831)
('sportsteam,O', 0.7894736842105263)
('company,O', 0.7209302325581395)
('facility,O', 0.6310679611650486)
('geo-loc,O', 0.6274509803921569)
('person,O', 0.556390977443609)
